In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
anime_list = pd.read_csv("anime.csv")
ratings = pd.read_csv("rating.csv")

In [ ]:
anime_list

In [ ]:
ratings

In [ ]:
anime_list['genre'] = anime_list['genre'].str.replace(',',' ')
anime_list

In [ ]:
ratings_f = ratings.groupby('user_id').filter(lambda x: len(x)>=50)
ratings_f

In [ ]:
rating_list = ratings_f.anime_id.unique().tolist()

In [ ]:
anime_list.episodes.replace({'Unknown':np.nan},inplace=True)

In [ ]:
anime_list = anime_list[anime_list.anime_id.isin(rating_list)]
anime_list

In [ ]:
anime_list.dropna(inplace=True)
anime_list.isna().sum()

In [ ]:
anime_list.duplicated(subset=['anime_id']).sum()

In [ ]:
anime_list.reset_index(drop=True,inplace=True)
anime_list

In [ ]:
import re
def text_cleaning(text):
    text = re.sub(r'&quot;', '', text)
    text = re.sub(r'.hack//', '', text)
    text = re.sub(r'&#039;', '', text)
    text = re.sub(r'A&#039;s', '', text)
    text = re.sub(r'I&#039;', 'I\'', text)
    text = re.sub(r'&amp;', 'and', text)
    text = re.sub(r"[^a-zA-Z0-9 ]","",text)
    
    return text

In [ ]:
anime_list['name'] = anime_list['name'].apply(text_cleaning)
anime_list

In [ ]:
Mapping_list = dict(zip(anime_list.name.tolist(),anime_list.anime_id.tolist()))

In [ ]:
mixed = pd.merge(anime_list,ratings_f,on='anime_id',how='left')
mixed

In [ ]:
mixed.rename(columns={'rating_x':'average_rating','rating_y':'user_rating'},inplace=True)
mixed.head()

In [ ]:
mixed.user_rating.replace({-1:np.nan},inplace=True)

In [ ]:
mixed.dropna(inplace=True)

In [ ]:
mixed.shape

In [ ]:
mixed.reset_index(drop=True,inplace=True)
mixed

In [ ]:
mixed.fillna("",inplace=True)
mixed = pd.DataFrame(mixed.groupby('anime_id')['type'].apply(lambda x: "%s" % ' '.join(x)))
mixed

In [ ]:
final = pd.merge(anime_list,mixed,on='anime_id',how='left')
final['metadata'] = final[['type_x','genre', 'episodes']].apply(lambda x:' '.join(x),axis=1)
final[['anime_id','name','metadata']]

In [ ]:
del mixed

In [ ]:
final.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(final['metadata'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(),index=final.index.tolist())
tfidf_df.shape

In [ ]:
tfidf_df

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components = 10)
latent_matrix = svd.fit_transform(tfidf_df)
explained = svd.explained_variance_ratio_.cumsum()
plt.plot(explained,'.-',ms=16,color='red')
plt.xlabel('Singular value components', fontsize=12)
plt.ylabel('Cumulative percent of variance', fontsize=12)
plt.show()

In [ ]:
n=10
latent_matrix_1 = pd.DataFrame(latent_matrix[:,0:n],index=final.name.tolist())

In [ ]:
mixed_1 = pd.merge(anime_list.anime_id,ratings_f,on='anime_id',how='left')
mixed_1.shape

In [ ]:
ratings_2 = pd.pivot_table(mixed_1,index=['anime_id'],columns=['user_id'],values=['rating'],aggfunc='sum').fillna(0).astype(int)

In [ ]:
ratings_2.replace({-1:0},inplace=True)
del mixed_1
ratings_2.to_csv('ratings_2.csv')
ratings_2.head(10)

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components = 135)
latent_matrix = svd.fit_transform(ratings_2)
explained = svd.explained_variance_ratio_.cumsum()
plt.plot(explained,'.-',ms=16,color='red')
plt.xlabel('Singular value components', fontsize=12)
plt.ylabel('Cumulative percent of variance', fontsize=12)
plt.show()

In [ ]:
n= 135
latent_matrix_2 = pd.DataFrame(latent_matrix[:,0:n],index = final.name.tolist())

In [ ]:
latent_matrix_1.to_csv("latent_1.csv")
latent_matrix_2.to_csv("latent_2.csv")

In [ ]:
latent_matrix_1 = pd.read_csv("latent_1.csv",index_col=0)
latent_matrix_2 = pd.read_csv("latent_2.csv",index_col=0)

In [ ]:
latent_matrix_1.head()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
a_1 = np.array(latent_matrix_1.loc['Mousou Dairinin']).reshape(1,-1)
a_2 = np.array(latent_matrix_2.loc['Mousou Dairinin']).reshape(1,-1)

score_1 = cosine_similarity(latent_matrix_1,a_1).reshape(-1)
score_2 = cosine_similarity(latent_matrix_2,a_2).reshape(-1)

hybrid = (score_1 + score_2)/2.0
dictdf = {'content':score_1,'collaborative':score_2,'hybrid':hybrid}
similar = pd.DataFrame(dictdf,index=latent_matrix_1.index)

similar.sort_values('hybrid',ascending=False,inplace=True)
similar[1:].head(11)